In [171]:
'''import cv2
import numpy as np
from skimage import io, color
import matplotlib.pyplot as plt
%matplotlib inline


def get_sift_feature(im):
    """
    Extract the SIFT features from an input image.

    Parameters:
        im (numpy.ndarray): An input image in float32 format with shape (H, W, 3).

    Returns:
        A 1D array of the mean of the SIFT descriptors. If no descriptors are found,
        returns a zero array of length 128.
    """
    gray_im = color.rgb2gray(im)
    sift = cv2.SIFT_create()
    _, descriptors = sift.detectAndCompute((gray_im * 255).astype("uint8"), None)
    if descriptors is None:
        return np.zeros((128,))
    return np.mean(descriptors, axis=0)



img = cv2.imread("/Users/aatmiya/Desktop/HKU/Year 2 Sem 2/COMP3314/Assignments/A3/comp3314-assignment-3-image-classification/test_ims/d6bae8f.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)


img1 = io.imread("/Users/aatmiya/Desktop/HKU/Year 2 Sem 2/COMP3314/Assignments/A3/comp3314-assignment-3-image-classification/test_ims/d6bb10e.jpg")
io.imshow(img1)
io.show()
print(get_sift_feature(img1))'''

'import cv2\nimport numpy as np\nfrom skimage import io, color\nimport matplotlib.pyplot as plt\n%matplotlib inline\n\n\ndef get_sift_feature(im):\n    """\n    Extract the SIFT features from an input image.\n\n    Parameters:\n        im (numpy.ndarray): An input image in float32 format with shape (H, W, 3).\n\n    Returns:\n        A 1D array of the mean of the SIFT descriptors. If no descriptors are found,\n        returns a zero array of length 128.\n    """\n    gray_im = color.rgb2gray(im)\n    sift = cv2.SIFT_create()\n    _, descriptors = sift.detectAndCompute((gray_im * 255).astype("uint8"), None)\n    if descriptors is None:\n        return np.zeros((128,))\n    return np.mean(descriptors, axis=0)\n\n\n\nimg = cv2.imread("/Users/aatmiya/Desktop/HKU/Year 2 Sem 2/COMP3314/Assignments/A3/comp3314-assignment-3-image-classification/test_ims/d6bae8f.jpg")\nimg = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)\nplt.imshow(img)\n\n\nimg1 = io.imread("/Users/aatmiya/Desktop/HKU/Year 2 Sem 2/COMP

In [172]:
import os
import cv2
import numpy as np
import pandas as pd
from skimage import io, color
from skimage.transform import resize
from skimage.feature import hog
#from sklearn.feature_extraction import image
#from sklearn.feature_extraction.image import extract_surf_features
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

# Get the directory path
directory_path = "/Users/aatmiya/Desktop/HKU/Year 2 Sem 2/COMP3314/Assignments/A3/data/train_ims"

# List filenames in alphabetical order
filenames = sorted(os.listdir(directory_path))

In [173]:
def get_sift_feature(im):
    gray_im = color.rgb2gray(im)
    sift = cv2.SIFT_create()
    _, descriptors = sift.detectAndCompute((gray_im * 255).astype("uint8"), None)
    if descriptors is None:
        return np.zeros((128,))
    return np.mean(descriptors, axis=0) 

def get_orb_feature(im):
    gray_im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    orb = cv2.ORB_create()
    _, descriptors = orb.detectAndCompute(gray_im, None)

    if descriptors is None:
        return np.zeros((32,))

    # Calculate the mean of the ORB descriptors
    mean_descriptor = np.mean(descriptors, axis=0)
    return mean_descriptor

def get_hog_feature(im):
    gray_im = color.rgb2gray(im)
    hog_features = hog(gray_im, orientations=4, pixels_per_cell=(4, 4), cells_per_block=(1, 1))
    hog_features = hog_features / np.sum(np.abs(hog_features))
    return(hog_features)

In [174]:
initial_images = []
train_df = pd.read_csv("/Users/aatmiya/Desktop/HKU/Year 2 Sem 2/COMP3314/Assignments/A3/data/train.csv") #change path
labels = train_df["label"].to_numpy()[:10000] #sliced to 1000


for i in range(10000): #only checking first 1000 images
    image = io.imread(f"{directory_path}/{filenames[i]}")
    initial_images.append(image)

In [175]:
sift_features = []
for image in initial_images:
    sift_features.append(get_sift_feature(image)) #checking for sift feature instead

sift_features = np.array(sift_features)
labels = np.array(labels)

scaler = MinMaxScaler(copy=False)
scaler.fit(sift_features)
sift_features_scaled = scaler.transform(sift_features)

print(sift_features_scaled)

pca = PCA(n_components=128, copy=False)
pca.fit(sift_features_scaled)
sift_features_reduced = pca.transform(sift_features_scaled)


param_grid = {'C': [0.01, 0.1, 1, 5], 'kernel': ['linear', 'rbf'], 'gamma':[0.2, 0.4, 0.5, 0.6]}
svc = SVC()
clf = GridSearchCV(svc, param_grid, cv=4)

# Train the SVM classifier
clf.fit(sift_features_reduced, labels)

# Print the best hyperparameters and score
print("Best parameters:", clf.best_params_)
print("Best score:", clf.best_score_)

[[0.32990313 0.33990147 0.25       ... 0.02678571 0.03095238 0.1845238 ]
 [0.01977401 0.05287356 0.37152779 ... 0.51388888 0.22777778 0.05555555]
 [0.17391305 0.17091454 0.13586956 ... 0.14356884 0.06376812 0.16485507]
 ...
 [0.02619414 0.03197492 0.1155303  ... 0.1751894  0.00909091 0.02045455]
 [0.36723163 0.2137931  0.39583333 ... 0.02662037 0.02407407 0.01574074]
 [0.19752281 0.12864722 0.16987179 ... 0.10817307 0.08301282 0.0573718 ]]
Best parameters: {'C': 1, 'gamma': 0.5, 'kernel': 'rbf'}
Best score: 0.308


In [176]:
hog_features = []
for image in initial_images:
    hog_features.append(get_hog_feature(image))

hog_features = np.array(hog_features)

scaler.fit(hog_features)
hog_features_scaled = scaler.transform(hog_features)

param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf'], 'gamma':[0.1, 0.2]}
clf2 = GridSearchCV(svc, param_grid, cv=5)

# Train the SVM classifier
clf2.fit(hog_features_scaled, labels)

# Print the best hyperparameters and score
print("Best parameters:", clf2.best_params_)
print("Best score:", clf2.best_score_)

Best parameters: {'C': 10, 'gamma': 0.2, 'kernel': 'rbf'}
Best score: 0.4243


In [178]:
combined_features = np.concatenate((sift_features_scaled, hog_features_scaled), axis = 1)

clf3 = GridSearchCV(svc, param_grid, cv=4)

clf3.fit(combined_features, labels)
print("Best parameters:", clf3.best_params_)
print("Best score:", clf3.best_score_)

Best parameters: {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'}
Best score: 0.461
